In [42]:
import numpy as np
import heapq
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from icecream import ic

class Node():
    def __init__(self,parent=None, position=None):
        self.parent=parent
        self.position=position
        self.g=0
        self.h=0
        self.f=0

    def __eq__(self,other):
        return self.position==other.position

    def __lt__(self, other):
        return self.f< other.f


In [43]:

def a_star(start, goal, grid):
    ic(f"using a* to figure shortest distance bw {start} and {goal}")
    start_node = Node(parent=None, position=tuple(start))
    goal_node = Node(parent=None, position=tuple(goal))
    open_list = []
    closed_list = []
    heapq.heappush(open_list, (start_node.f, start_node))

    while open_list:
        current_node = heapq.heappop(open_list)[1]
        if current_node == goal_node: # implicitly comparing x and y coordinates of both nodes
            return current_node.g

        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            if node_position[0] > (len(grid) - 1) or node_position[0] < 0 or node_position[1] > (len(grid[0]) - 1) or node_position[1] < 0:
                continue
            if grid[node_position[0]][node_position[1]] != 0:
                continue

            new_node = Node(current_node, node_position)
            children.append(new_node)

        for child in children:
            if child in closed_list:
                continue

            child.g = current_node.g + 1
            child.h = abs(child.position[0] - goal_node.position[0]) + abs(child.position[1] - goal_node.position[1])
            child.f = child.g + child.h

            if any(child == open_node[1] and child.g > open_node[1].g for open_node in open_list):
                continue
            heapq.heappush(open_list, (child.f, child))

        closed_list.append(current_node)

    return float('inf')


In [44]:

def create_distance_matrix(waypoints, grid):
    n = len(waypoints)
    distance_matrix = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                distance_matrix[i][j] = a_star(waypoints[i], waypoints[j], grid)
    return distance_matrix


In [47]:

def solve_tsp(distance_matrix):
    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        # Convert routing variable indices to distance matrix indices.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]
        
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

    solution = routing.Solve()
    if solution:
        index = routing.Start(0)
        plan_output = 'Route:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        print(plan_output)
        print('Route distance: {}'.format(route_distance))


In [48]:

# Example grid (0 represents free path and 1 represents obstacle)
grid_size = 10
grid = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
ic(grid)

# Define waypoints as tuples (x, y) within the grid bounds
waypoints = [(0, 0), (2, 5), (5, 2), (7, 8)]

# Create distance matrix using A* for each pair of waypoints
distance_matrix = create_distance_matrix(waypoints, grid)
ic(grid)
ic (distance_matrix)
# Solve TSP
solve_tsp(distance_matrix)

ic| grid: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
ic| f"using a* to figure shortest distance bw {start} and {goal}": 'using a* to figure shortest distance bw (0, 0) and (2, 5)'
ic| f"using a* to figure shortest distance bw {start} and {goal}": 'using a* to figure shortest distance bw (0, 0) and (5, 2)'
ic| f"using a* to figure shortest distance bw {start} and {goal}": 'using a* to figure shortest distance bw (0, 0) and (7, 8)'
ic| f"using a* to figure shortest distance bw {start} and {goal}": 'using a* to figure shortest distance bw (2, 5) and (0, 0)'
ic| f"using a* to figure shortest distance bw {start} and {goa

Route:
 0 -> 2 -> 3 -> 1 -> 0

Route distance: 30
